In [ ]:
"""
This script can be used as skeleton code to read the challenge train and test
"""
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [246]:
## Read csvs
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

In [247]:
## Handle missing values
train_df.fillna('NA', inplace=True)
test_df.fillna('NA', inplace=True)

days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Training Data Total
new_train_df = train_df.copy()
new_train_df = new_train_df.drop(['date', 'bcced'], axis=1)

# Testing Data
new_test_df = test_df.copy()
new_test_df = new_test_df.drop(['date', 'bcced'], axis=1)

In [249]:
# Label Encoder
def LabelEnc(data_df, title_column):
    lab_Enc = LabelEncoder()
    new_data_df=data_df.copy()
    new = lab_Enc.fit_transform(new_data_df[title_column].astype(str))
    new_data_df[title_column] = new
    return new_data_df[title_column]

In [250]:
## Function for processing data
def processing(dataset, new_dataset):

    for (index, row) in dataset['date'].items():

        # Process for date
        date_list = row.split(' ')
        if len(date_list) > 4:
            day = date_list[0][0:3]
            month = date_list[2]
            hour = float(date_list[4][0:2])
            minutes = float(date_list[4][3:5]) / 60
            new_hour = hour + minutes
            new_dataset.at[index, 'day'] = str(day)
            new_dataset.at[index, 'month'] = str(month)
            new_dataset.at[index, 'hour'] = new_hour

        # Process for mail_type
        new_dataset.at[index, 'mail_type'] = new_dataset.at[index, 'mail_type'].lower()
        new_dataset.at[index, 'org'] = new_dataset.at[index, 'org'].lower()
        new_dataset.at[index, 'tld'] = new_dataset.at[index, 'tld'].lower()
        
    #Label Encoder to transform categories into numbers
    categorical=list(new_dataset)
    for cat in categorical:
        new_dataset[cat]=LabelEnc(new_dataset,cat)

    return new_dataset

In [251]:
## Preprocessing on Train & Test sets
new_train_df = processing(train_df, new_train_df)
new_test_df = processing(test_df, new_test_df)

In [252]:
# Training Labels
trainingLabels_df= new_train_df[['label']]
# Training Data
trainingData_df=new_train_df.copy()
trainingData_df=trainingData_df.drop(['label'],axis=1)

In [253]:
## Save into csv files
new_train_df.to_csv("Training_Data_Labels.csv", index=True, index_label='Id')
trainingData_df.to_csv("Training_Data.csv", index=True, index_label='Id')
trainingLabels_df.to_csv("Training_Labels.csv", index=True, index_label='Id')
new_test_df.to_csv("Testing_Data.csv", index=True, index_label='Id')